# Deep Learning project - Dataset Preprocessing
### Ludovico Comito - Matr. 1837155
### Giulio Fedeli - Matr. 1873677
### Lorenzo Cirone - Matr. 1930811

## Dataset preprocessing
The source dataset we use for our experiments is MS Marco Document ranking dataset. All the original files to download can be found inside [this github repo](https://github.com/microsoft/msmarco/blob/master/Datasets.md) under the "document ranking dataset" paragraph.
The msmarco-docs.tsv file (download [here](https://msmarco.z22.web.core.windows.net/msmarcoranking/msmarco-docs.tsv.gz)) is a 22gb file containing the all the dataset's documents and contains the columns [docid, url, title, body]. \\
Each split (train, validation and test) is essentially made of two files: one tsv containing queries and their corresponding query id (qid) and one top 100 file that maps each query id to 100 ranked docids. \\
As the original dataset contains millions of documents, we decided to extract 8k train queries, 2k validation queries and 2k test queries. In orther to further reduce the number of documents to load, we reduce the rankings to the first 10 relevant documents instead of the first 100.
\\
The final datasets will look like this:


*   A training dataset that contains all the documents to be indexed plus the training queries (to train the model in a multitask fashion).
*   Validation and Test datasets containing the queries and the file that maps queries to the ids of the first 10 relevant documents (labels).








As the training corpus is very large (22gb), we will use the dask library that allows to load the corpus as a dataframe and split it  in chunks that can fit into memory.

In [ ]:
!pip install dask

In [ ]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
import pickle
import torch

### Step 1: load and sample queries

In [ ]:
# load queries files and rename the columns
train_queries_path = 'msmarco-doctrain-queries.tsv'
train_queries = pd.read_csv(train_queries_path, sep = '\t')
train_queries.columns = ['qid', 'query']

val_queries_path = 'msmarco-docdev-queries.tsv'
val_queries = pd.read_csv(val_queries_path, sep = '\t')
val_queries.columns = ['qid', 'query']

test_queries_path = 'docleaderboard-queries.tsv'
test_queries = pd.read_csv(test_queries_path, sep = '\t')
test_queries.columns = ['qid', 'query']

In [ ]:
# Sample 8k queries for train, 2k for val and 2k for validation
train_queries = train_queries.sample(8000, random_state = 42).reset_index(drop = True)
val_queries = val_queries.sample(2000, random_state = 42).reset_index(drop = True)
test_queries = test_queries.sample(2000, random_state = 42).reset_index(drop = True)

print(f'train queries: {train_queries.shape[0]}')
print(f'val queries: {val_queries.shape[0]}')
print(f'test queries: {val_queries.shape[0]}')

### Step 2: take the corresponding ranked documents
The top100 documents file contain the top 100 documents ranked for each query according to their relevance with the query. The document at rank 1 is most relevant to the query and the document at 100 is least relevant among the 100 documents. Since we have already reduced the number of queries, let’s reduce this one too.

In [ ]:
# Load top 100 tsvs
train_top_100_path = 'dataset/train/msmarco-doctrain-top100'

train_top100 = pd.read_table(train_top_100_path, delimiter=' ', header = None)
train_top100.columns = ['qid', 'Q0', 'docid', 'rank', 'score', 'runstring']
print('Shape before resizing=>',train_top100.shape)
train_top100.head()

# Reducing train_top100 for training
training_ranked100 = train_top100[train_top100['qid'].isin(train_queries['qid'].unique())].reset_index(drop=True)
print('Shape after resizing=>', training_ranked100.shape)
training_ranked100.head()

In [ ]:
val_top_100_path = 'dataset/val/msmarco-docdev-top100'

val_top100 = pd.read_table(val_top_100_path, delimiter = ' ',header=None)
val_top100.columns = ['qid','Q0','docid','rank','score','runstring']
print('Shape before resizing=>',val_top100.shape)
val_top100.head()

val_ranked100 = val_top100[val_top100['qid'].isin(val_queries['qid'].unique())].reset_index(drop=True)
print('Shape after resizing=>',val_ranked100.shape)
val_ranked100.head()

In [ ]:
test_top_100_path = 'dataset/test/docleaderboard-top100.tsv'

test_top100 = pd.read_table(test_top_100_path, delimiter = ' ',header=None)
test_top100.columns = ['qid','Q0','docid','rank','score','runstring']
print('Shape before resizing=>',test_top100.shape)
test_top100.head()

test_ranked100 = test_top100[test_top100['qid'].isin(test_queries['qid'].unique())].reset_index(drop=True)
print('Shape after resizing=>',test_ranked100.shape)
test_ranked100.head()

In [ ]:
def reduce_ranked_documents(ranking_document, queries):
    '''
    Filters out only the queries that have been subsamples and takes
    the first 10 document rankings for each query.
    '''
    # filter out the rankings for queries that are not in the current query set
    print('Original shape=>',ranking_document.shape)
    reduced_ranked_documents = ranking_document[ranking_document['qid'].isin(queries['qid'].unique())].reset_index(drop=True)
    print('Shape after filtering=>',ranking_document.shape)

    # take the first 10 documents for each query
    rel = list(range(1,11))
    reduced_ranked_documents['rel'] = reduced_ranked_documents['rank'].apply(lambda x: 1 if x in rel else np.nan)
    reduced_result=reduced_ranked_documents.dropna()

    print('Shape after reduction=>', reduced_result.shape)


    return reduced_result

In [ ]:
print('Reducing train_top100')
train_top_10 = reduce_ranked_documents(train_top100, train_queries)

print('Reducing val_top100')
val_top_10 = reduce_ranked_documents(val_top100, val_queries)

print('Reducing test_top100')
test_top_10 = reduce_ranked_documents(test_top100, test_queries)

Save pickled splits

In [ ]:
train_queries_path = 'processed_train_queries.pkl'
train_top_10_path = 'train_top_10.pkl'

val_queries_path = 'processed_val_queries.pkl'
val_top_10_path = 'processed_val_ranked_top100.pkl'

test_queries_path = 'processed_test_queries.pkl'
test_top_10_path = 'processed_test_ranked_top100.pkl'

train_queries.to_pickle(train_queries_path)
training_ranked100.to_pickle(train_top_10_path)

val_queries.to_pickle(val_queries_path)
val_ranked100.to_pickle(val_top_10_path)

test_queries.to_pickle(test_queries_path)
test_ranked100.to_pickle(test_top_10_path)

### Step 3: reduce corpus
Take only the relevant documents from the main 22GB file

In [ ]:
df = dd.read_table('dataset/msmarco-docs.tsv', blocksize = 100e6, header = None)
df.columns = ['docid','url','title','body']
df.head()

In [ ]:
def create_corpus(result, df):
    '''
    Filters out the documents present in the split.
    '''
    unique_docid = result['docid'].unique()
    condition = df['docid'].isin(unique_docid)
    corpus = df[condition].reset_index(drop = True)
    corpus = corpus.drop(columns = 'url')
    print('Number of Rows=>',len(corpus))
    return corpus

In [ ]:
training_corpus = create_corpus(train_top_10, df)
training_corpus_df = training_corpus.compute()
training_corpus_df.to_pickle('training_corpus_df.pkl')

val_corpus = create_corpus(val_top_10, df)
val_corpus_df = val_corpus.compute()
val_corpus_df.to_pickle('val_corpus_df.pkl')

test_corpus = create_corpus(test_top_10, df)
test_corpus.head()
test_corpus_df = test_corpus.compute()
test_corpus_df.to_pickle('test_corpus_df.pkl')

### Step 4: Semantic clustering
As pointed out by the DSI paper, reassigning IDs based on semantic clustering (documents within the same cluster will have similar docids) can greatly improve the performance of the model, expecially when paired with beam search as a decoding strategy. \\
The following code implements semantic clustering by first creating the embeddings for each document and applying a k-means algorithm to clusterize and assign semantic ids.

In order to create the embeddings for our documents, we utilize a sentence transformer from the HuggingFace library. In particular, we utilize *distilroberta-base-nli-matryoshka-256* which is a distilled Roberta based model that we found being a good tradeoff between the dimensionality of the produced embeddings (256 dimensions vs 768 of BERT) and the quality of the obtained representations.




In [ ]:
!pip install sentence_transformers

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
files_dict = {
    'train_corpus_path': 'training_corpus_df.pkl', # train corpus to be embedded
    'val_corpus_path': 'val_corpus_df.pkl', # val corpus to be embedded
    'test_corpus_path': 'test_corpus_df.pkl' # test corpus to be embedded

}

In [ ]:
def load_pickle(filepath):
    '''
    Loads and returns a pickle file stored in filepath.
    '''
    with open(filepath, 'rb') as file:
        loaded_file = pickle.load(file)
        return loaded_file


def export_pickle(element, filepath):
    '''
    Stores the passed elements as a pickle file.
    '''
    with open(filepath, 'wb') as file:
        pickle.dump(element, file)

In [ ]:
train_corpus_df = load_pickle(files_dict['train_corpus_path'])
val_corpus_df = load_pickle(files_dict['val_corpus_path'])
test_corpus_df = load_pickle(files_dict['test_corpus_path'])

In [ ]:
# We unify the title and body of each documents for all the splits.
train_corpus_df['document'] = train_corpus_df['title'] + ' ' + train_corpus_df['body']
train_unified_df = train_corpus_df[['docid', 'document']]

val_corpus_df['document'] = val_corpus_df['title'] + ' ' + val_corpus_df['body']
val_unified_df = val_corpus_df[['docid', 'document']]

test_corpus_df['document'] = test_corpus_df['title'] + ' ' + test_corpus_df['body']
test_unified_df = test_corpus_df[['docid', 'document']]

In [ ]:
# load the sentence transformer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
sentence_transformer = SentenceTransformer('tomaarsen/distilroberta-base-nli-matryoshka-256').to(device)

In [ ]:
def create_embeddings_dict(corpus_df, sentence_transformer):
    '''
    Given a corpus (a dataframe with docid and body columns), this function
    utilizes the sentence transformer to embed the document and store it
    inside a dictionary with its corresponding key being the original docid.
    '''
    embeddings_dict = {}

    for i in range(len(corpus_df)):
        current_docid = corpus_df['docid'].iloc[i]
        current_document = corpus_df['body'].iloc[i]

        if type(current_document) is str:
            embedding = sentence_transformer.encode(current_document)
            embeddings_dict[current_docid] = embedding
            torch.cuda.empty_cache()

    return embeddings_dict

In [ ]:
# create dictionaries mapping docid to embeddings for train, val and test splits
train_embeddings_dict = create_embeddings_dict(train_corpus_df)
val_embeddings_dict = create_embeddings_dict(val_corpus_df)
test_embeddings_dict = create_embeddings_dict(test_corpus_df)

In [ ]:
# merge all the embeddings into a single dict
total_embeddings_dict = {}

total_embeddings_dict.update(train_embeddings_dict)
total_embeddings_dict.update(val_embeddings_dict)
total_embeddings_dict.update(test_embeddings_dict)

In [ ]:
# export a pickle checkpoint containing all the embeddings
export_pickle(total_embeddings_dict, 'total_embeddings_dict.pkl')

Here we implement the algorithm that clusters and reassigns the docids: the algorithm recursively creates clusters of embeddings using k-means. The final result will be a dictionary that maps the original docids to the newly created semantic ids.

In [ ]:
from sklearn.cluster import KMeans


def cluster_documents(docs_embeddings):
    '''
    Clusters documents into k clusters using k-means on the embeddings.
    '''
    embeddings = list(docs_embeddings.values())
    kmeans = KMeans(n_clusters = 10, random_state = 0).fit(embeddings)
    clusters = {i: [] for i in range(10)}
    for docid, label in zip(docs_embeddings.keys(), kmeans.labels_):
        clusters[label].append(docid)
    return clusters


def generate_semantic_ids(docs_embeddings, c = 100, prefix = ''):
    '''
    Recursively generates semantically structured identifiers with mapping.
    '''
    if len(docs_embeddings) == 0:
        return {}

    # Cluster the documents based on their embeddings
    clusters = cluster_documents(docs_embeddings)
    new_doc_ids = {}

    for i in range(10):
        cluster_docids = clusters[i]
        cluster_embeddings = {docid: docs_embeddings[docid] for docid in cluster_docids}

        if len(cluster_embeddings) > c:
            # Recursively cluster further
            Jrest = generate_semantic_ids(cluster_embeddings, c, prefix = f"{prefix}{i}")
        else:
            # Assign unique identifier within this cluster
            Jrest = {docid: f"{prefix}{i}{j}" for j, docid in enumerate(cluster_docids)}

        new_doc_ids.update(Jrest)

    return new_doc_ids

In [ ]:
docid_to_semantic_map = generate_semantic_ids(total_embeddings_dict)

In [ ]:
export_pickle(docid_to_semantic_map, 'docid_to_semantic_map.pkl')

Finally, we use the map function of the Pandas library to map every docid in the dataset to its corresponding semantic id.

In [ ]:
def map_docid_to_semantic(docid):
    return docid_to_semantic_map.get(docid, None)


In [ ]:
# Map docids to semantic ids and create a new column
mapped_train_top_10 = train_top_10.copy()
mapped_train_top_10['semantic_id'] = mapped_train_top_10['docid'].map(map_docid_to_semantic)

mapped_val_top_10 = val_top_10.copy()
mapped_val_top_10['semantic_id'] = mapped_val_top_10['docid'].map(map_docid_to_semantic)

mapped_test_top_10 = test_top_10.copy()
mapped_test_top_10['semantic_id'] = mapped_test_top_10['docid'].map(map_docid_to_semantic)

At training time, the model will need to index all the corpuses of train, validation and test, while it will be trained only on the training queries.

In [ ]:
# Remap docids of the various corpuses and unify them into a single dataset
mapped_train_corpus = train_corpus_df.copy()
mapped_train_corpus['semantic_id'] = mapped_train_corpus['docid'].map(map_docid_to_semantic)

mapped_val_corpus = val_corpus_df.copy()
mapped_val_corpus['semantic_id'] = mapped_val_corpus['docid'].map(map_docid_to_semantic)

mapped_test_corpus = test_corpus_df.copy()
mapped_test_corpus['semantic_id'] = mapped_test_corpus['docid'].map(map_docid_to_semantic)

full_corpus_df = pd.concat([mapped_train_corpus, mapped_val_corpus, mapped_test_corpus], ignore_index=True)

As a final step, it's time to process the queries. Inside the training dataset, we will add to the corpus the train queries, where each query will have as a label the id of the document ranked as number one. \\
However, for validation and test, we want to evaluate the capability of the model at producing a ranking of 10 relevant documents for that query. In that case, we will aim at making a dataframe where each query is mapped to the 10 most relevant ids.

In [ ]:
# Extract the top 1 ranked id for each train query, and create a dictionary that maps each query to the docid.
query_to_semantic_docid_map = {}
for i in range(len(train_queries)):
    current_row = train_queries.iloc[i]
    current_qid = current_row['qid']
    current_query = current_row['query']

    current_semantic_docid_top10 = train_top10[train_top10['qid'] == current_qid]
    if len(current_semantic_docid_top10) > 0:
        current_semantic_docid = current_semantic_docid_top10.iloc[0]['semantic_id'] # take the docid with rank 1
        query_to_semantic_docid_map[current_query] = current_semantic_docid

# transpose the dictionary
transposed_dict = {'document': list(query_to_semantic_docid_map.keys()), 'semantic_id': list(query_to_semantic_docid_map.values())}
# Convert the transposed dictionary to a DataFrame
train_queries_df = pd.DataFrame(transposed_dict)

full_corpus_df['doctype'] = 'document'
train_queries_df['doctype'] = 'query'

# in order to save memory, take just the first 50 words from each document
def shorten_document(row):
    document = row['document']
    words = document.split()
    max_words = min(50, len(words))
    document = ' '.join(words[:max_words])

  return document

full_corpus_df['document'] = full_corpus_df.apply(shorten_document, axis = 1)

# the final training dataset will be a merge of all the documents to be indexed and the train queries
train_df = pd.concat([full_corpus_df, train_queries_df], ignore_index = True)

# shuffle the questions and documents in the dataset
train_df = train_df.sample(frac = 1).reset_index(drop = True)

# make sure that there are no null values or duplicates
train_df = train_df.dropna()
train_df = train_df.drop_duplicates

export_pickle(train_df, 'train_df.pkl')

In our proposed Query Generation approach, queries are generated using the *all-t5-base-v1* model from HuggingFace. Each query is then concatenated to the original document.\
 We only want to generate queries for documents, not for the already existing training queries, hence we will work on the full_corpus_df and then merge it afain with the train_queries_df.

In [ ]:
from transformers import pipeline

# initialize query generation pipeline
query_pipe = pipeline('text2text-generation', model = 'doc2query/all-t5-base-v1')

In [ ]:
def append_query(row):
    '''
    Generates a query for each row of the document and appends it at the beginning.
    '''
    doc = row['document']
    query = query_pipe(doc)[0]['generated_text']
    return query + ' ' + doc

full_corpus_df['document'] = full_corpus_df.apply(append_query, axis=1)

In [ ]:
train_df_qg = pd.concat([full_corpus_df, train_queries_df], ignore_index = True)
export_pickle(train_df, 'train_df_qg.pkl')